##  Import Libraries

In [1]:
import pystac
import fsspec
import xarray as xr
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm

## Data Load

### Extract Terra Climate Catalogue

In [2]:
# Extracting data from Planetary Computer Terra Climate catalog
url = "https://planetarycomputer.microsoft.com/api/stac/v1/collections/terraclimate"
collection = pystac.read_file(url)
asset = collection.assets["zarr-https"]
store = fsspec.get_mapper(asset.href)
ds = xr.open_zarr(store, **asset.extra_fields["xarray:open_kwargs"])
ds

<xarray.Dataset>
Dimensions:                 (time: 744, lat: 4320, lon: 8640, crs: 1)
Coordinates:
  * crs                     (crs) int16 3
  * lat                     (lat) float64 89.98 89.94 89.9 ... -89.94 -89.98
  * lon                     (lon) float64 -180.0 -179.9 -179.9 ... 179.9 180.0
  * time                    (time) datetime64[ns] 1958-01-01 ... 2019-12-01
Data variables: (12/18)
    aet                     (time, lat, lon) float32 ...
    def                     (time, lat, lon) float32 ...
    pdsi                    (time, lat, lon) float32 ...
    pet                     (time, lat, lon) float32 ...
    ppt                     (time, lat, lon) float32 ...
    ppt_station_influence   (time, lat, lon) float32 ...
    ...                      ...
    tmin                    (time, lat, lon) float32 ...
    tmin_station_influence  (time, lat, lon) float32 ...
    vap                     (time, lat, lon) float32 ...
    vap_station_influence   (time, lat, lon) float32 ...
    vpd                     (time, lat, lon) float32 ...
    ws                      (time, lat, lon) float32 ...

### Load Frog Presence Data

In [3]:
path_frog_presence_aus = "Data\BWW Challenge 2022 Level 3 Frog_Occurence_Australia.csv"
df_frog_aus=pd.read_csv(path_frog_presence_aus,usecols=['eventDate','year','month','decimalLatitude','decimalLongitude','occurrenceStatus',"stateProvince"])

print(df_frog_aus.shape)
df_frog_aus["eventDate"] = pd.to_datetime(df_frog_aus["eventDate"])
df_frog_aus.head()

(125621, 7)


,occurrenceStatus,eventDate,year,month,stateProvince,decimalLatitude,decimalLongitude
0,PRESENT,2017-11-14,2017,11,Vic,-38.1,144.6
1,PRESENT,2017-11-25,2017,11,Qld,-26.7,153.1
2,PRESENT,2018-01-03,2018,1,Nsw,-30.3,152.9
3,PRESENT,2018-01-26,2018,1,Nt,-12.6,131.1
4,PRESENT,2018-02-06,2018,2,Qld,-26.3,152.9


In [4]:
path_frog_presence_SA_CR = "Data\BWW Challenge 2022 Level 3 Frog_Occurence_South_Africa_and_Costa_Rica.csv"
df_frog_SA_CR=pd.read_csv(path_frog_presence_SA_CR,usecols=['eventDate','year','month','decimalLatitude','decimalLongitude','occurrenceStatus',"stateProvince"])

print(df_frog_SA_CR.shape)
df_frog_SA_CR["eventDate"] = pd.to_datetime(df_frog_SA_CR["eventDate"])
df_frog_SA_CR.head()

(6670, 7)


,occurrenceStatus,eventDate,year,month,stateProvince,decimalLatitude,decimalLongitude
0,PRESENT,2018-01-07 14:59:00,2018,1,Limón,10.438840,-83.786274
1,PRESENT,2017-11-06 08:31:00,2017,11,Mpumalanga,-24.926687,31.476465
2,PRESENT,2019-08-04 03:22:00,2019,8,Puntarenas,9.389377,-84.142044
3,PRESENT,2018-01-15 21:15:00,2018,1,Limón,10.440689,-83.785854
4,PRESENT,2018-01-04 14:22:00,2018,1,Limón,10.440703,-83.786194


### Grid based approach for extracting frog count 
#### For calculating the frog density over Australia, we will create a grid of 225 sq. kms then we will evaluate the frog presence points on each of the grid

### Australia region bbox

In [5]:
# Australia region bbox(excluding "Tasmania" region)
aus = {"type":"Polygon",
       "coordinates":[[[153.896484,-39.327584],[113.062499,-39.327584],
                       [113.062499,-10.521216],[153.896484,-10.521216],[153.896484,-39.327584]]]}
aus["coordinates"] = aus["coordinates"][0][0:4]
min_longi = min(aus["coordinates"])[0]
max_longi = max(aus["coordinates"])[0]
min_lati =  min(aus["coordinates"])[1]
max_lati = max(aus["coordinates"])[1]

print("min_lati = ",min_lati,"\n","min_longi = ",min_longi,"\n","max_lati = ",max_lati,"\n","max_longi = ",max_longi)

min_lati =  -39.327584 
 min_longi =  113.062499 
 max_lati =  -10.521216 
 max_longi =  153.896484


In [6]:
## Australia region bbox for "Tasmania" region
tas = {"type":"Polygon",
       "coordinates":[[[148.541748,-39.398856],[143.802246,-39.398856],
                       [143.802246,-43.69331],[148.541748,-43.69331],[148.541748,-39.398856]]]}
tas["coordinates"] = tas["coordinates"][0][0:4]

min_longi_tas = min(tas["coordinates"])[0]
max_longi_tas = max(tas["coordinates"])[0]
min_lati_tas =  min(tas["coordinates"])[1]
max_lati_tas = max(tas["coordinates"])[1]

print("min_lati = ",min_lati_tas,"\n","min_longi = ",min_longi_tas,"\n","max_lati = ",max_lati_tas,"\n","max_longi = ",max_longi_tas)

min_lati =  -43.69331 
 min_longi =  143.802246 
 max_lati =  -39.398856 
 max_longi =  148.541748


In [7]:
# Combining the whole of Australia including Tasmania region 
aus_whole = {"min_lati":min_lati_tas,"max_lati":max_lati,"min_longi":min_longi,"max_longi":max_longi}

### Grid formation

In [8]:
#Using grid based approach, creating 225 sq.kms grid, the approx. difference between lat-lon for 225 sq.kms area : lat is 0.1331 and lon is 0.15365 respectively
#66500 grids formulated for the entire region of Australia
bbox_grid_whole = [({"min_x":np.round(x,4), "min_y":np.round(y,4), "max_x":np.round(x + 0.15365,4),
                     "max_y":np.round(y + .1331,4)}) for x, y in itertools.product(np.arange(aus_whole["min_longi"], aus_whole["max_longi"],0.15365),
                                                                                   np.arange(aus_whole["min_lati"], aus_whole["max_lati"],.1331))]
print(len(bbox_grid_whole))

66500


In [10]:
# Calculate the frog count (greater than 0) by iterating through each of the grid and subsetting frog presence data, 
#append to a dictionary with bounding box coordinates and shape of the subset data as - frog count
filt_lat = {}
i=1
for _,bbox in tqdm(enumerate(bbox_grid_whole)):  
    longi_lati_df_rang = df_frog_aus[((df_frog_aus['decimalLongitude'] >= bbox["min_x"]) & (df_frog_aus['decimalLongitude'] <= bbox["max_x"])) & 
                           ((df_frog_aus['decimalLatitude'] >= bbox["min_y"]) & (df_frog_aus['decimalLatitude'] <=bbox["max_y"]))]
    if longi_lati_df_rang.shape[0]>0:
        filt_lat[i] ={}
        filt_lat[i]["coord"] = bbox
        filt_lat[i]["frog_count"] = longi_lati_df_rang.shape[0]
        i=i+1
aus_whole_filt_cord = filt_lat

66500it [02:16, 486.48it/s]


In [11]:
# Sample dictionary with bounding box and frog count
dict(list(aus_whole_filt_cord.items())[0:2])

{1: {'coord': {'min_x': 113.6771,
   'min_y': -24.9262,
   'max_x': 113.8307,
   'max_y': -24.7931},
  'frog_count': 1},
 2: {'coord': {'min_x': 114.138,
   'min_y': -27.7213,
   'max_x': 114.2917,
   'max_y': -27.5882},
  'frog_count': 8}}

In [13]:
# Converting the frog_id dictionary to dataframe having 2660 rows & 6 columns
aus_whole_filt_cord_df = pd.DataFrame.from_dict(aus_whole_filt_cord,orient="index")

aus_whole_filt_cord_df["min_lon"] = [i["min_x"] for i in aus_whole_filt_cord_df["coord"]]
aus_whole_filt_cord_df["min_lat"] = [i["min_y"] for i in aus_whole_filt_cord_df["coord"]]

aus_whole_filt_cord_df["max_lon"] = [i["max_x"] for i in aus_whole_filt_cord_df["coord"]]
aus_whole_filt_cord_df["max_lat"] = [i["max_y"] for i in aus_whole_filt_cord_df["coord"]]


print(aus_whole_filt_cord_df.shape)
aus_whole_filt_cord_df.head()


(2660, 6)


,coord,frog_count,min_lon,min_lat,max_lon,max_lat
1,"{'min_x': 113.6771, 'min_y': -24.9262, 'max_x'...",1,113.6771,-24.9262,113.8307,-24.7931
2,"{'min_x': 114.138, 'min_y': -27.7213, 'max_x':...",8,114.1380,-27.7213,114.2917,-27.5882
3,"{'min_x': 114.599, 'min_y': -28.9192, 'max_x':...",3,114.5990,-28.9192,114.7526,-28.7861
4,"{'min_x': 114.599, 'min_y': -28.7861, 'max_x':...",28,114.5990,-28.7861,114.7526,-28.6530
5,"{'min_x': 114.599, 'min_y': -28.653, 'max_x': ...",35,114.5990,-28.6530,114.7526,-28.5199


In [14]:
# Selecting time frame based on frogid dataset
ds_date = ds.sel(time = slice("2017-11-01","2019-11-01"))

# filtering data for Austrlia region based on coordinates
ds_aus = ds_date.where((ds.lat>=aus_whole["min_lati"]) & (ds.lat<=aus_whole["max_lati"]) & 
                       ((ds.lon>=aus_whole["min_longi"] ) & (ds.lon<=aus_whole["max_longi"])),drop = True)

#  Converting the xarray format to pandas dataframe 
ds_aus = ds_aus.to_dataframe().reset_index()

ds_aus["time"] = pd.to_datetime(ds_aus["time"])

KeyboardInterrupt: 